# Data Generator
## Meilenstein 3: Logischer und Physischer Entwurf
### Gruppe 30
### Boiko Bohdana, Alexander Grath

_Version: 0.1_

# Datengenerierung (Data-Generator mit Jupyter Notebook)

## Mit **UniVie VPN** verbinden und Datenbankverbindung herstellen.

In [1]:
import oracledb
import atexit
import random
import string
from datetime import date, timedelta
import time

The Oracle Instant Client comes prepackaged in the dbs_python.zip.
So we just need to call it to use python-oracledb in **Thick** mode.

In [2]:
# I am using an ARM based Mac, I have downloaded the basic package from https://www.oracle.com/database/technologies/instant-client/macos-arm64-downloads.html
# I kept the provided package as is, I added a new folder "instantclient" to hold the client required to run on my system
path_to_client = "instantclient"
oracledb.init_oracle_client(lib_dir=path_to_client)

In [3]:
username="###"  #db user name
password="###" #db password 
con_string="oracle19.cs.univie.ac.at:1521/orclcdb"

Using `with` would be best practice but in order to allow this Jupyter Notebook to be split up into multiple cells we have to just open the connection and keep it as a variable.
Using [atexit](https://docs.python.org/3/library/atexit.html) in an effort to prevent ungracefull shutowns/disconnects.

In [4]:
connection = oracledb.connect(user=username,password=password,dsn=con_string)
cursor = connection.cursor()

def close_connection():
    print("Stopping kernel... closing database connection.")
    try:
        if cursor:
            cursor.close()
        if connection:
            connection.close()
        print("Database connection closed cleanly.")
    except oracledb.Error:
        pass

atexit.register(close_connection)

<function __main__.close_connection()>

## Datenbankenstruktur

Um den Code mehrmals laufen lassen zu können werden alle alten Tables gedropt

In [5]:
def delete_database(cursor, statement):
    try:
        cursor.execute(statement)
    except oracledb.Error as e:
        error_obj, = e.args
        if error_obj.code != 942:
            print(f"Fehler bei: {statement[:50]} {e}")

drop_statements = [
    "DROP TABLE Ticket_Linie",
    "DROP TABLE Ticket",
    "DROP TABLE Kunde", 
    "DROP TABLE Stadt",
    "DROP TABLE Linie_Haltestelle",
    "DROP TABLE Haltestelle",
    "DROP TABLE Linie",
    "DROP TABLE Wartung",
    "DROP TABLE Bus",
    "DROP TABLE Zug",
    "DROP TABLE Strassenbahn",
    "DROP TABLE Fahrzeug",
    "DROP TABLE Fahrer",
    "DROP TABLE Kontrolleur",
    "DROP TABLE Personal"
]

for statement in drop_statements:
    delete_database(cursor, statement)

Jetzt können die Tables generiert werden.

In [6]:
create_statements = [
    """CREATE TABLE Fahrzeug (
         FahrzeugID INTEGER,
         Baujahr    SMALLINT NOT NULL,
         Kapazitaet INTEGER,
         CONSTRAINT PK_Fahrzeug PRIMARY KEY (FahrzeugID),
         CONSTRAINT CC_Fahrzeug_Kapazitaet CHECK (Kapazitaet > 0)
       )""",
    """CREATE TABLE Personal (
         PersonalNr       INTEGER,
         Name             VARCHAR2(50) NOT NULL,
         Gehalt           NUMERIC(10,2),
         Einstellungsdatum DATE NOT NULL,
         CONSTRAINT PK_Personal PRIMARY KEY (PersonalNr),
         CONSTRAINT CC_Personal_Gehalt CHECK (Gehalt > 0)
       )""",
    """CREATE TABLE Haltestelle (
         HaltestellenID INTEGER,
         Name           VARCHAR2(50) NOT NULL,
         Barrierefrei   CHAR(1) DEFAULT 'N',
         CONSTRAINT PK_Haltestelle PRIMARY KEY (HaltestellenID),
         CONSTRAINT UQ_Haltestelle_Name UNIQUE (Name),
         CONSTRAINT CC_Haltestelle_Barrierefrei CHECK (Barrierefrei IN ('J','N'))
       )""",
    """CREATE TABLE Linie (
         LinienNr      INTEGER,
         Taktung       INTEGER,
         Betriebsbeginn TIMESTAMP NOT NULL,
         Betriebsende   TIMESTAMP NOT NULL,
         CONSTRAINT PK_Linie PRIMARY KEY (LinienNr),
         CONSTRAINT CC_Linie_Taktung CHECK (Taktung > 0)
       )""",
    """CREATE TABLE Stadt (
         PLZ      VARCHAR2(4) NOT NULL,
         Ort      VARCHAR2(50) NOT NULL,
         CONSTRAINT PK_Stadt PRIMARY KEY (PLZ)
       )""",
    """CREATE TABLE Bus (
         FahrzeugID  INTEGER,
         Kennzeichen VARCHAR2(20) NOT NULL,
         Niederflur  CHAR(1) DEFAULT 'N',
         CONSTRAINT PK_Bus PRIMARY KEY (FahrzeugID),
         CONSTRAINT UQ_Bus_Kennzeichen UNIQUE (Kennzeichen),
         CONSTRAINT FK_Bus_Fahrzeug FOREIGN KEY (FahrzeugID)
           REFERENCES Fahrzeug(FahrzeugID)
           ON DELETE CASCADE,
         CONSTRAINT CC_Bus_Niederflur CHECK (Niederflur IN ('J','N'))
       )""",
    """CREATE TABLE Zug (
         FahrzeugID  INTEGER,
         Baureihe    VARCHAR2(30) NOT NULL,
         Wagonanzahl INTEGER,
         CONSTRAINT PK_Zug PRIMARY KEY (FahrzeugID),
         CONSTRAINT FK_Zug_Fahrzeug FOREIGN KEY (FahrzeugID)
           REFERENCES Fahrzeug(FahrzeugID)
           ON DELETE CASCADE,
         CONSTRAINT CC_Zug_Wagon CHECK (Wagonanzahl > 0)
       )""",
    """CREATE TABLE Strassenbahn (
         FahrzeugID  INTEGER,
         Spurweite   INTEGER NOT NULL,
         Stromsystem VARCHAR2(30) NOT NULL,
         CONSTRAINT PK_Strassenbahn PRIMARY KEY (FahrzeugID),
         CONSTRAINT FK_Strassenbahn_Fahrzeug FOREIGN KEY (FahrzeugID)
           REFERENCES Fahrzeug(FahrzeugID)
           ON DELETE CASCADE
       )""",
    """CREATE TABLE Wartung (
         WartungID  INTEGER,
         Datum      DATE NOT NULL,
         Kosten     NUMERIC(10,2),
         Art        VARCHAR2(50) NOT NULL,
         FahrzeugID INTEGER NOT NULL,
         CONSTRAINT PK_Wartung PRIMARY KEY (WartungID),
         CONSTRAINT FK_Wartung_Fahrzeug FOREIGN KEY (FahrzeugID)
           REFERENCES Fahrzeug(FahrzeugID),
         CONSTRAINT CC_Wartung_Kosten CHECK (Kosten >= 0)
       )""",
    """CREATE TABLE Fahrer (
         PersonalNr         INTEGER,
         Fuehrerscheinklasse VARCHAR2(10) NOT NULL,
         MentorNr           INTEGER,
         CONSTRAINT PK_Fahrer PRIMARY KEY (PersonalNr),
         CONSTRAINT FK_Fahrer_Personal FOREIGN KEY (PersonalNr)
           REFERENCES Personal(PersonalNr)
           ON DELETE CASCADE,
         CONSTRAINT FK_Fahrer_Mentor FOREIGN KEY (MentorNr)
           REFERENCES Fahrer(PersonalNr)
           ON DELETE SET NULL
       )""",
    """CREATE TABLE Kontrolleur (
         PersonalNr          INTEGER,
         Zustaendigkeitsbereich VARCHAR2(50) NOT NULL,
         CONSTRAINT PK_Kontrolleur PRIMARY KEY (PersonalNr),
         CONSTRAINT FK_Kontrolleur_Personal FOREIGN KEY (PersonalNr)
           REFERENCES Personal(PersonalNr)
           ON DELETE CASCADE
       )""",
    """CREATE TABLE Linie_Haltestelle (
         LinienNr       INTEGER,
         HaltestellenID INTEGER,
         Reihenfolge    INTEGER,
         CONSTRAINT PK_Linie_Haltestelle PRIMARY KEY (LinienNr, HaltestellenID),
         CONSTRAINT FK_Linie_Haltestelle_Linie FOREIGN KEY (LinienNr)
           REFERENCES Linie(LinienNr)
           ON DELETE CASCADE,
         CONSTRAINT FK_Linie_Haltestelle_Haltestelle FOREIGN KEY (HaltestellenID)
           REFERENCES Haltestelle(HaltestellenID)
           ON DELETE CASCADE,
         CONSTRAINT CC_LH_Reihenfolge CHECK (Reihenfolge > 0)
       )""",
    """CREATE TABLE Kunde (
         KundenID INTEGER,
         Name     VARCHAR2(50) NOT NULL,
         Email    VARCHAR2(100) NOT NULL,
         Strasse  VARCHAR2(50),
         Hausnr   VARCHAR2(10),
         PLZ      VARCHAR2(4), 
         CONSTRAINT PK_Kunde PRIMARY KEY (KundenID),
         CONSTRAINT UQ_Kunde_Email UNIQUE (Email),
         CONSTRAINT FK_Kunde_Stadt FOREIGN KEY (PLZ) REFERENCES Stadt(PLZ)
       )""",
    """CREATE TABLE Ticket (
         TicketID   INTEGER,
         Kaufdatum  DATE NOT NULL,
         Preis      NUMERIC(8,2),
         GueltigVon DATE NOT NULL,
         GueltigBis DATE NOT NULL,
         KundenID   INTEGER NOT NULL,
         CONSTRAINT PK_Ticket PRIMARY KEY (TicketID),
         CONSTRAINT FK_Ticket_Kunde FOREIGN KEY (KundenID)
           REFERENCES Kunde(KundenID), -- Hier fehlte vorher das Komma
         CONSTRAINT CC_Ticket_Preis CHECK (Preis > 0),
         CONSTRAINT CC_Ticket_Datum CHECK (GueltigBis >= GueltigVon)
       )""",
    """CREATE TABLE Ticket_Linie (
         TicketID INTEGER,
         LinienNr INTEGER,
         CONSTRAINT PK_Ticket_Linie PRIMARY KEY (TicketID, LinienNr),
         CONSTRAINT FK_Ticket_Line_Ticket FOREIGN KEY (TicketID)
           REFERENCES Ticket(TicketID)
           ON DELETE CASCADE,
         CONSTRAINT FK_Ticket_Line_Linie FOREIGN KEY (LinienNr)
           REFERENCES Linie(LinienNr)
           ON DELETE CASCADE
       )"""
]

for statement in create_statements:
    try:
        cursor.execute(statement)
    except oracledb.Error as e:
        print(f"Fehler bei: {statement[:30]} {e}")
        break

connection.commit()

#### Helferfunktionen um Daten zu generieren
[stackoverflow](https://stackoverflow.com/questions/2257441/random-string-generation-with-upper-case-letters-and-digits) zur Hilfe genommen.

## Data Generator

In [7]:
def random_string(length):
    return ''.join(random.choices(string.ascii_letters, k=length))

def random_date(start_year=2020, end_year=2026):
    start = date(start_year, 1, 1)
    end = date(end_year, 12, 31)
    return start + timedelta(days=random.randint(0, (end - start).days))

Zuerst werden Städte generiert um Kunden zu erstellen.

In [8]:
cities_count = 100

cities = []
plz_list = []
for i in range(cities_count):
    plz = f"{1000 + i}"
    ort = f"Stadt_{random_string(5)}"
    cities.append((plz, ort))
    plz_list.append(plz)

Danach werden sie in die DB eingetragen.

In [9]:
cursor.executemany("INSERT INTO Stadt (PLZ, Ort) VALUES (:1, :2)", cities)
connection.commit()

Jetzt können die Kunden generiert werden.

In [10]:
customer_count = 2500

customers = []
customer_ids = []
for index in range(1, customer_count + 1):
    customer_id = index
    name = f"Kunde_{random_string(6)}"
    email = f"user_{index}_{random_string(3)}@example.com"
    street = f"Str. {random.randint(1, 200)}"
    home = f"{random.randint(1, 100)}"
    plz = random.choice(plz_list)
    
    customers.append((customer_id, name, email, street, home, plz))
    customer_ids.append(customer_id)

Die Kunden werden in die DB eingetragen.

In [11]:
cursor.executemany("INSERT INTO Kunde (KundenID, Name, Email, Strasse, Hausnr, PLZ) VALUES (:1, :2, :3, :4, :5, :6)", customers)
connection.commit()

Jetzt generieren wir die Fahrzeuge.

In [12]:
bus_count = 100
train_count = 100
tram_count = 100
vehicle_count = bus_count + train_count + tram_count

vehicles = []
busses = []
trains = []
trams = []

for index in range(1, vehicle_count + 1):
    vehicle_id = index
    construction_year = random.randint(1990, 2020)
    capacity = random.randint(50, 300)
    vehicles.append((vehicle_id, construction_year, capacity))
    
    if index <= bus_count: # busses
        registration = f"A-{random_string(3).upper()}-{index}"
        low_floor = random.choice(['J', 'N'])
        busses.append((vehicle_id, registration, low_floor))
    elif index <= bus_count + tram_count: # trains
        series = f"ICE-{random.randint(1, 4)}"
        wagons = random.randint(3, 12)
        trains.append((vehicle_id, series, wagons))
    else: # trams
        spur = 1435
        strom = "600V"
        trams.append((vehicle_id, spur, strom))

Die Fahrzeuge in die DB eintragen.

In [13]:
cursor.executemany("INSERT INTO Fahrzeug (FahrzeugID, Baujahr, Kapazitaet) VALUES (:1, :2, :3)", vehicles)
cursor.executemany("INSERT INTO Bus (FahrzeugID, Kennzeichen, Niederflur) VALUES (:1, :2, :3)", busses)
cursor.executemany("INSERT INTO Zug (FahrzeugID, Baureihe, Wagonanzahl) VALUES (:1, :2, :3)", trains)
cursor.executemany("INSERT INTO Strassenbahn (FahrzeugID, Spurweite, Stromsystem) VALUES (:1, :2, :3)", trams)
connection.commit()

Jetzt noch die Tickets generieren.

In [14]:
ticket_count = 5000

tickets = []
for index in range(1, ticket_count + 1):
    ticket_id = index
    buying_date = random_date(2020, 2024)
    price = round(random.uniform(10.0, 200.0), 2)
    _from = buying_date
    to = buying_date + timedelta(days=2)
    customer_id = random.choice(customer_ids)
    
    tickets.append((ticket_id, buying_date, price, _from, to, customer_id))

Die Tickets in die DB.

In [15]:
cursor.executemany("INSERT INTO Ticket (TicketID, Kaufdatum, Preis, GueltigVon, GueltigBis, KundenID) VALUES (:1, :2, :3, :4, :5, :6)", tickets)
connection.commit()

Testen ob das ganze auch funktioniert hat.

In [16]:
tables_to_check = ['Stadt', 'Kunde', 'Fahrzeug', 'Ticket']
for t in tables_to_check:
    cursor.execute(f"SELECT COUNT(*) FROM {t}")
    count = cursor.fetchone()[0]
    print(f"Anzahl Datensätze in {t}: {count}")

Anzahl Datensätze in Stadt: 100
Anzahl Datensätze in Kunde: 2500
Anzahl Datensätze in Fahrzeug: 300
Anzahl Datensätze in Ticket: 5000


#### Schritt 0:
Angabe durchlesen. Hier habe ich realisiert dass wir nicht unsere Datenbank auffüllen müssen sondern auch beliebige Werte erstellen hätten können.
Die *_count Variablen wurden angepasst um _"das Datenvolumen muss eine realistische Größenordnung aufweisen"_ zu erfüllen und die Zusammenhänge der Daten ist auch gegeben.

## Queries
Aus der Angabe soll jede Anfrage 2 dieser Konstrukte haben: Having, Group By, All und Subselect.

### Query #1
Findet Kunden, deren Gesamtausgaben für Tickets höher sind als der Durchschnitt aller Ticket-Einnahmen pro Kunde um ihnen einen Goldschienen-Schlüsselanhänger zu schicken.

In [17]:
# Konstrukte: GROUP BY, HAVING, SUBSELECT
goldschinenfahrer = """
SELECT k.Name, SUM(t.Preis) as Gesamtausgaben
FROM Kunde k
JOIN Ticket t ON k.KundenID = t.KundenID
GROUP BY k.Name
HAVING SUM(t.Preis) > (
    SELECT AVG(SummeProKunde)
    FROM (
        SELECT SUM(Preis) as SummeProKunde 
        FROM Ticket 
        GROUP BY KundenID
    )
)
"""
cursor.execute(goldschinenfahrer)
for row in cursor.fetchmany(5):
    print(row)

('Kunde_wVMCyx', 400.72)
('Kunde_BaQIIj', 348.75)
('Kunde_TdPBzI', 493.4)
('Kunde_NuqdSh', 396.21)
('Kunde_biYNgn', 333.06)


### Query #2
Findet Tickets, deren Preis höher ist als alle anderen Tickets, die vor 01.01.2024 gekauft wurden.

In [18]:
# Konstrukte: SUBSELECT, ALL
expensive_tickets = """
SELECT t.TicketID, t.Preis, t.Kaufdatum, k.Name
FROM Ticket t
JOIN Kunde k ON t.KundenID = k.KundenID
WHERE t.Preis > ALL (
    SELECT Preis 
    FROM Ticket 
    WHERE Kaufdatum < TO_DATE('2024-01-01', 'YYYY-MM-DD')
)
"""

cursor.execute(expensive_tickets)
for row in cursor.fetchmany(5):
    print(row)

### Query #3
Durchschnittsalter der Fahrzeuge.

In [19]:
# Konstrukte: GROUP BY, SUBSELECT
average_age = """
SELECT FahrzeugTyp, AVG(Baujahr) as Durchschnittsjahr
FROM (
    SELECT f.Baujahr, 'Bus' as FahrzeugTyp 
    FROM Fahrzeug f 
    JOIN Bus b ON f.FahrzeugID = b.FahrzeugID
    UNION ALL
    SELECT f.Baujahr, 'Zug' as FahrzeugTyp 
    FROM Fahrzeug f 
    JOIN Zug z ON f.FahrzeugID = z.FahrzeugID
    UNION ALL
    SELECT f.Baujahr, 'Strassenbahn' as FahrzeugTyp 
    FROM Fahrzeug f 
    JOIN Strassenbahn s ON f.FahrzeugID = s.FahrzeugID
)
GROUP BY FahrzeugTyp
"""
cursor.execute(average_age)
results = cursor.fetchall()

for row in results:
    print(row)

('Bus', 2004.99)
('Zug', 2007.02)
('Strassenbahn', 2005.33)


## Update und Delete Stored Procedure.

Eine Stored Procedure soll Tupel aus einer Tabelle aktualisieren.

Ticket wird verlängert, der Preis steigt und die Gültigkeit herhöht sich.

In [ ]:
proc_update_sql = """
CREATE OR REPLACE PROCEDURE TicketVerlaengern(
    p_TicketID IN INTEGER, 
    p_PreisAufschlag IN NUMBER
) IS
BEGIN
    UPDATE Ticket
    SET Preis = Preis + p_PreisAufschlag,
        GueltigBis = GueltigBis + 7
    WHERE TicketID = p_TicketID;
    COMMIT;
END;
"""
cursor.execute(proc_update_sql)

In [ ]:
ticket = cursor.execute("SELECT Preis, GueltigBis FROM Ticket WHERE TicketID=1").fetchone()
print("\nVor Update Ticket:", ticket)

ticket_id = 1
markup = 3.50
cursor.callproc("TicketVerlaengern", [ticket_id, markup])
ticket = cursor.execute("SELECT Preis, GueltigBis FROM Ticket WHERE TicketID = :id", id=ticket_id).fetchone()
print("Nach Update Ticket:", ticket)


Vor Update Ticket: (25.05, datetime.datetime(2021, 3, 27, 0, 0))
Nach Update Ticket 1: (28.55, datetime.datetime(2021, 4, 3, 0, 0))


Eine andere soll Tupel einer Tabelle löschen.

Die abgelaufenen Tickets eines Kunden werden gelöscht.

In [ ]:
proc_delete_sql = """
CREATE OR REPLACE PROCEDURE DeleteOldTickets(
    p_KundenID IN INTEGER,
    p_Stichtag IN DATE
) IS
BEGIN
    DELETE FROM Ticket
    WHERE KundenID = p_KundenID 
    AND GueltigBis < p_Stichtag;
    COMMIT;
END;
"""
cursor.execute(proc_delete_sql)

In [ ]:


# Test Delete: Alte Tickets von Kunde 5 löschen
# (Wir simulieren ein Datum in der Zukunft als Stichtag, damit sicher was gelöscht wird)
kunden_id_test = 5
stichtag_test = date(2030, 1, 1) 
cursor.callproc("DeleteOldTickets", [kunden_id_test, stichtag_test])
print(f"Lösch-Prozedur für Kunde {kunden_id_test} ausgeführt.")